# <font color="red">**3. Without FS - Fine-tuning Hyperparameters**</font>

**Author:** Osmar Bolivar

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import plotly.graph_objects as go

## **1. Monthly data**

In [2]:
raw = pd.read_csv("./Data/FINAL_DATASET_G12.csv", index_col=0)
raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5901 entries, 2010-01-01 to 2024-10-31
Columns: 461 entries, ipc_all to freq
dtypes: float64(460), object(1)
memory usage: 20.8+ MB


In [3]:
df = raw.copy().query('freq == "month"')
df.drop(columns=["freq"], inplace=True)
df

,ipc_all,lag_1,lag_2,lag_3,lag_6,lag_9,lag_12,beef_lp,beef_cb,beef_sc,...,inflacion,inflacion en bolivia,inflación bolivia,ipc,la inflacion,la inflación,pib,pib bolivia,que es inflacion,que es pib
2010-01-31,0.074472,0.263790,0.457498,0.784931,1.447299,5.323961,11.041182,16.000000,16.925806,15.702581,...,47.0,99.0,185.0,133.0,128.0,139.0,45.0,71.0,260.0,54.0
2010-02-28,0.312381,0.074472,0.263790,0.457498,1.417620,3.200128,8.128586,16.000000,17.132143,15.690000,...,72.0,100.0,182.0,135.0,120.0,136.0,42.0,108.0,251.0,54.0
2010-03-31,0.685510,0.312381,0.074472,0.263790,0.643999,2.117642,6.563669,16.000000,16.858065,15.650000,...,60.0,101.0,178.0,136.0,111.0,133.0,129.0,146.0,242.0,54.0
2010-04-30,1.215574,0.685510,0.312381,0.074472,0.784931,1.447299,5.323961,16.000000,16.400000,15.650000,...,85.0,101.0,175.0,140.0,103.0,130.0,97.0,125.0,233.0,54.0
2010-05-31,1.384706,1.215574,0.685510,0.312381,0.457498,1.417620,3.200128,16.000000,16.000000,15.650000,...,80.0,102.0,171.0,145.0,102.0,127.0,118.0,84.0,224.0,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-30,3.844356,3.516994,3.460018,3.063085,2.118325,2.847826,2.725282,24.016667,22.500000,22.000000,...,39.0,55.0,62.0,54.0,49.0,58.0,48.0,29.0,59.0,58.0
2024-07-31,3.953707,3.844356,3.516994,3.460018,1.863150,2.075432,2.700872,23.859355,22.693548,21.919355,...,41.0,51.0,79.0,50.0,60.0,50.0,42.0,30.0,67.0,50.0
2024-08-31,5.192869,3.953707,3.844356,3.516994,2.516604,1.594145,3.058496,24.212903,23.419355,21.967742,...,42.0,39.0,70.0,55.0,69.0,111.0,51.0,39.0,68.0,42.0
2024-09-30,6.182067,5.192869,3.953707,3.844356,3.063085,2.118325,2.847826,24.668000,23.525000,22.250000,...,28.0,53.0,49.0,58.0,59.0,59.0,49.0,39.0,64.0,38.0


In [4]:
isna = df.isna().sum()
isna[isna > 0]

Series([], dtype: int64)

In [5]:
df.shape

(178, 460)

## **2. Train and Val sets**

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [7]:
train_size = int(len(df) * 0.8)
train, validation = df[:train_size], df[train_size:]
print(f'Obs in train set: {train.shape[0]}; variables in train set: {train.shape[1]}')
print(f'Obs in validation set: {validation.shape[0]}; variables in validation set: {validation.shape[1]}')

Obs in train set: 142; variables in train set: 460
Obs in validation set: 36; variables in validation set: 460


In [8]:
validation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 2021-11-30 to 2024-10-31
Columns: 460 entries, ipc_all to que es pib
dtypes: float64(460)
memory usage: 129.7+ KB


In [9]:
scaler = StandardScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(train), columns=train.columns, index=train.index)
validation_scaled = pd.DataFrame(scaler.transform(validation), columns=validation.columns, index=validation.index)
#test_scaled = pd.DataFrame(scaler.transform(test), columns=test.columns, index=test.index)

X_train = train_scaled.drop('ipc_all', axis=1)
#y_train = train_scaled['ipc_all']
y_train = train['ipc_all']

X_validation = validation_scaled.drop('ipc_all', axis=1)
#y_validation = validation_scaled['ipc_all']
y_validation = validation['ipc_all']

#X_test = test_scaled.drop('ipc_all', axis=1)

## **3. Algorithms**

In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

tscv5 = TimeSeriesSplit(n_splits=5)

### **3.1. Ridge**   

**Without tuning (Validation):**
MSE:  18.579; R2:  -7.406; MAE:  3.843

In [37]:
#ridge = Ridge(random_state=0)
ridge = Ridge(alpha=65.8, fit_intercept=True, positive=False, max_iter=10000, random_state=0) 
# Fit on training set
ridge.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
ridge_train_pred = ridge.predict(X_train)
ridge_val_pred = ridge.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_ridge = mean_squared_error(y_train, ridge_train_pred)
r2_train_ridge = r2_score(y_train, ridge_train_pred)
mae_train_ridge = mean_absolute_error(y_train, ridge_train_pred)
print("Train MSE: ", mse_train_ridge)
print("Train R2: ", r2_train_ridge)
print("Train MAE: ", mae_train_ridge)
# Calculate Forecast metrics on validation set
mse_val_ridge = mean_squared_error(y_validation, ridge_val_pred)
r2_val_ridge = r2_score(y_validation, ridge_val_pred)
mae_val_ridge = mean_absolute_error(y_validation, ridge_val_pred)
print("Validation MSE: ", mse_val_ridge)
print("Validation R2: ", r2_val_ridge)
print("Validation MAE: ", mae_val_ridge)

Train MSE:  0.055639641662979376
Train R2:  0.992051925702896
Train MAE:  0.17384793485563171
Validation MSE:  14.169605277377443
Validation R2:  -5.411386609101433
Validation MAE:  3.424983534363479


In [ ]:
# Define parameter grid for GridSearchCV
alphas_ridge = np.logspace(0.1, 5, num=500)
#alphas_ridge = np.arange(1.0, 5, 0.1)
param_grid = {
    'alpha': alphas_ridge,  # [1.0] 
    'fit_intercept': [True],  # [True, False] 
    'copy_X': [True], 
    'max_iter': [10000], 
    'tol': [0.0001], 
    'solver': ['auto'], 
    'positive': [False],  # [True, False]
    'random_state': [0]    
}

# Instantiate Ridge model
ridge = Ridge()
# Define GridSearchCV object
grid_search_ridge = GridSearchCV(ridge, param_grid, cv=tscv5, scoring='neg_mean_squared_error') 
# Fit GridSearchCV on training set
grid_search_ridge.fit(X_train, y_train)
# Print best parameter and score from GridSearchCV
print("Best parameter: ", grid_search_ridge.best_params_)
print("Best score: ", -grid_search_ridge.best_score_)
# Make predictions on validation set using best model from GridSearchCV
best_ridge = grid_search_ridge.best_estimator_
ridge_val_pred = best_ridge.predict(X_validation)
# Calculate Forecast metrics on validation set
mse_val_ridge = mean_squared_error(y_validation, ridge_val_pred)
r2_val_ridge = r2_score(y_validation, ridge_val_pred)
mae_val_ridge = mean_absolute_error(y_validation, ridge_val_pred)
print("Validation MSE: ", mse_val_ridge)
print("Validation R2: ", r2_val_ridge)
print("Validation MAE: ", mae_val_ridge)

Best parameter:  {'alpha': 65.83197338664691, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'positive': False, 'random_state': 0, 'solver': 'auto', 'tol': 0.0001}
Best score:  1.9914801689143915
Validation MSE:  14.168694774993714
Validation R2:  -5.410974629891262
Validation MAE:  3.42488540218311


c:\Users\osmar.bolivar\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [38]:
# Get the coefficients from the Ridge model
coef = ridge.coef_
# Create a dataframe of feature importances
feature_importance_ridge = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
# Sort the features by importance
feature_importance_ridge = feature_importance_ridge.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_ridge.columns = ['feat_ridge', 'imp_ridge']
# Print the feature importances
feature_importance_ridge.head(15)

,feat_ridge,imp_ridge
0,lag_1,0.125405
1,watermelon_su,0.091952
2,lag_2,0.090207
3,noodle_co,0.073528
4,platano_co,0.069566
5,chicken_co,0.069496
6,chicken_tj,0.066633
7,lag_3,0.066498
8,chicken_tr,0.061202
9,sugar_cb,0.059402


### **3.2. Lasso**
Without tuning:
Validation MSE:  0.796
Validation R2:  0.640
Validation MAE:  0.689

In [41]:
#lasso = Lasso(random_state=0)
lasso = Lasso(alpha=0.12, fit_intercept=True, max_iter=10000, random_state=0)  
# Fit on training set
lasso.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
lasso_train_pred = lasso.predict(X_train)
lasso_val_pred = lasso.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_lasso = mean_squared_error(y_train, lasso_train_pred)
r2_train_lasso = r2_score(y_train, lasso_train_pred)
mae_train_lasso = mean_absolute_error(y_train, lasso_train_pred)
print("Train MSE: ", mse_train_lasso)
print("Train R2: ", r2_train_lasso)
print("Train MAE: ", mae_train_lasso)
# Calculate Forecast metrics on validation set
mse_val_lasso = mean_squared_error(y_validation, lasso_val_pred)
r2_val_lasso = r2_score(y_validation, lasso_val_pred)
mae_val_lasso = mean_absolute_error(y_validation, lasso_val_pred)
print("Validation MSE: ", mse_val_lasso)
print("Validation R2: ", r2_val_lasso)
print("Validation MAE: ", mae_val_lasso)

Train MSE:  0.24445203474427432
Train R2:  0.9650802399843905
Train MAE:  0.38000356740271735
Validation MSE:  0.4068738063847913
Validation R2:  0.8158999334996018
Validation MAE:  0.5066383943874704


In [ ]:
# Define parameter grid for GridSearchCV
alphas_lasso = np.logspace(-1, 2, num=500)
#alphas_lasso = np.arange(0.1, 100, 0.05)
param_grid = {
    'alpha': [0.4],  # alphas_lasso [1.0]
    'fit_intercept': [True], 
    'precompute': [False], 
    'copy_X': [True], 
    'max_iter': [10000],  # [1000]
    'tol': [0.0001], 
    'warm_start': [False], 
    'positive': [False], 
    'random_state': [0], 
    'selection': ['cyclic']
}

# Instantiate lasso model
lasso = Lasso()
# Define GridSearchCV object
grid_search_lasso = GridSearchCV(lasso, param_grid, cv=tscv5, scoring='neg_mean_squared_error')  
# Fit GridSearchCV on training set
grid_search_lasso.fit(X_train, y_train)
# Print best parameter and score from GridSearchCV
print("Best parameter: ", grid_search_lasso.best_params_)
print("Best score: ", -grid_search_lasso.best_score_)
# Make predictions on validation set using best model from GridSearchCV
best_lasso = grid_search_lasso.best_estimator_
y_val_pred = best_lasso.predict(X_validation)
# Calculate RMSE on validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)
# Make predictions on test set using best model from GridSearchCV
#y_test_pred_lasso = best_lasso.predict(X_test)

In [42]:
# Get the coefficients from the Lasso model
coef = lasso.coef_
# Create a dataframe of feature importances
feature_importance_lasso = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
# Sort the features by importance
feature_importance_lasso = feature_importance_lasso.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_lasso.columns = ['feat_lasso', 'imp_lasso']
# Print the feature importances
feature_importance_lasso.head(15)

,feat_lasso,imp_lasso
0,lag_1,2.057520
1,sugar_lp,0.237159
2,ycorn_tj,0.120583
3,tomato_cb,0.071688
4,chicken_tr,0.066120
5,peas_su,0.061409
6,oil_tr,0.058944
7,oil_su,0.041473
8,watermelon_po,0.005355
9,rice4_su,0.000000


### **3.3. ElasticNet**
Without tuning:
Validation MSE:  0.2237
Validation R2:  0.250
Validation MAE:  0.401

In [43]:
enet = ElasticNet(random_state=0)
#enet = ElasticNet(alpha=0.4, l1_ratio=0.97, fit_intercept=True, max_iter=10000, random_state=0) 
# Fit on training set
enet.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
enet_train_pred = enet.predict(X_train)
enet_val_pred = enet.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_enet = mean_squared_error(y_train, enet_train_pred)
r2_train_enet = r2_score(y_train, enet_train_pred)
mae_train_enet = mean_absolute_error(y_train, enet_train_pred)
print("Train MSE: ", mse_train_enet)
print("Train R2: ", r2_train_enet)
print("Train MAE: ", mae_train_enet)
# Calculate Forecast metrics on validation set
mse_val_enet = mean_squared_error(y_validation, enet_val_pred)
r2_val_enet = r2_score(y_validation, enet_val_pred)
mae_val_enet = mean_absolute_error(y_validation, enet_val_pred)
print("Validation MSE: ", mse_val_enet)
print("Validation R2: ", r2_val_enet)
print("Validation MAE: ", mae_val_enet)

Train MSE:  1.0217414468680504
Train R2:  0.8540451252125664
Train MAE:  0.7809637927541454
Validation MSE:  1.8824189747377202
Validation R2:  0.14825321022734783
Validation MAE:  1.298307329537221


In [ ]:
# Define parameter grid for GridSearchCV
#alphas_enet = np.logspace(-1, 2, num=500)
alphas_enet = np.arange(0.1, 10, 0.05)
param_grid = {
    'alpha': [1.0], # alphas_enet
    'l1_ratio': [0.5], # np.arange(0.1, 0.95, 0.05)
    'fit_intercept': [True], 
    'precompute': [False], 
    'max_iter': [10000], 
    'copy_X': [True], 
    'tol': [0.0001], 
    'warm_start': [False], 
    'positive': [False], 
    'random_state': [0], 
    'selection': ['cyclic']
}

# Instantiate enet model
enet = ElasticNet()
# Define GridSearchCV object
grid_search_enet = GridSearchCV(enet, param_grid, cv=tscv5, scoring='neg_mean_squared_error') 
# Fit GridSearchCV on training set
grid_search_enet.fit(X_train, y_train)
# Print best parameter and score from GridSearchCV
print("Best parameter: ", grid_search_enet.best_params_)
print("Best score: ", -grid_search_enet.best_score_)
# Make predictions on validation set using best model from GridSearchCV
best_enet = grid_search_enet.best_estimator_
y_val_pred = best_enet.predict(X_validation)
# Calculate RMSE on validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)
# Make predictions on test set using best model from GridSearchCV
#y_test_pred_enet = best_enet.predict(X_test)

In [44]:
# Get the coefficients from the enet model
coef = enet.coef_
# Create a dataframe of feature importances
feature_importance_enet = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
# Sort the features by importance
feature_importance_enet = feature_importance_enet.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_enet.columns = ['feat_enet', 'imp_enet']
# Print the feature importances
feature_importance_enet.head(15)

,feat_enet,imp_enet
0,lag_1,0.566793
1,lag_2,0.338107
2,lag_3,0.156339
3,chicken_tj,0.100170
4,sugar_lp,0.092091
5,sugar_bol,0.086308
6,sugar_tj,0.084150
7,chicken_co,0.076131
8,sugar_cb,0.073989
9,oil2_bol,0.071359


### **3.4. ADA**
Without tuning:
Validation MSE:  0.451
Validation R2:  0.796
Validation MAE:  0.577  (KEEP DEFAULT)

In [65]:
#ada = AdaBoostRegressor(random_state=0)
ada = AdaBoostRegressor(learning_rate=1.0, random_state=0)
# Fit the model on training data
ada.fit(X_train, y_train)
# Make predictions on the validation set
ada_train_pred = ada.predict(X_train)
ada_val_pred = ada.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_ada = mean_squared_error(y_train, ada_train_pred)
r2_train_ada = r2_score(y_train, ada_train_pred)
mae_train_ada = mean_absolute_error(y_train, ada_train_pred)
print("Train MSE: ", mse_train_ada)
print("Train R2: ", r2_train_ada)
print("Train MAE: ", mae_train_ada)
# Evaluate the model on the validation set
mse_val_ada = mean_squared_error(y_validation, ada_val_pred)
r2_val_ada = r2_score(y_validation, ada_val_pred)
mae_val_ada = mean_absolute_error(y_validation, ada_val_pred)
print("Validation MSE: ", mse_val_ada)
print("Validation R2: ", r2_val_ada)
print("Validation MAE: ", mae_val_ada)

Train MSE:  0.12605613379384767
Train R2:  0.9819929912009873
Train MAE:  0.2914050810960719
Validation MSE:  0.45148067007033915
Validation R2:  0.7957164602899327
Validation MAE:  0.5771463288334994


In [ ]:
# Define the AdaBoost Regressor
ada = AdaBoostRegressor()
# Define the range of hyperparameters to search over
param_grid_ada = {
    'estimator': [DecisionTreeRegressor(max_depth=3),  # [None]
                  DecisionTreeRegressor(max_depth=4),
                  DecisionTreeRegressor(max_depth=5),
                  DecisionTreeRegressor(max_depth=6),
                  DecisionTreeRegressor(max_depth=7),
                  DecisionTreeRegressor(max_depth=8) ],   
    'n_estimators': [50],  # range(50, 200, 5)
    'learning_rate': [0.99], # np.logspace(-5,1, 100)
    'loss': ['square'], # ['linear']
    'random_state': [0]
}

# Tune hyperparameters using GridSearchCV with TimeSeriesSplit
grid_search_ada = GridSearchCV(estimator=ada, param_grid=param_grid_ada, cv=tscv5, scoring='neg_mean_squared_error')
grid_search_ada.fit(X_train, y_train)
# Evaluate the model using the best hyperparameters on the test set
ada_best = AdaBoostRegressor(**grid_search_ada.best_params_)
ada_best.fit(X_train, y_train)
# Make predictions on the validation set
y_val_pred = ada_best.predict(X_validation)
# Evaluate the model on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)
# Print the best hyperparameters and the best score
print("Best parameters found: ", grid_search_ada.best_params_)
print("Lowest MSE found: ", -grid_search_ada.best_score_)
# Evaluate the model on the test set
#y_test_pred_ada = ada_best.predict(X_test)

Validation MSE:  0.3740074941755078
Validation R2:  -2.274570833078731
Validation MAE:  0.5144651864672598
Best parameters found:  {'estimator': DecisionTreeRegressor(max_depth=6), 'learning_rate': 0.002009233002565048, 'random_state': 123}
Lowest MSE found:  7.190820165324213


In [66]:
# Create a DataFrame with the feature importance values
feature_importance_ada = pd.DataFrame({'Feature': X_train.columns, 'Importance': ada.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_ada = feature_importance_ada.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_ada.columns = ['feat', 'imp_ada']
# Print the feature importance DataFrame
feature_importance_ada.head(15)

,feat,imp_ada
0,lag_1,0.570280
1,oil2_bol,0.122915
2,chicken_tj,0.052660
3,lard_bol,0.037283
4,chicken_bol,0.025443
5,flour_lp,0.021915
6,lemon_co,0.017326
7,veglard_tr,0.014727
8,sugar_lp,0.011180
9,lag_2,0.009337


### **3.5. GBR**
Without tuning:
Validation MSE:  1.170
Validation R2:  0.470
Validation MAE:  0.895

In [71]:
#gbr = GradientBoostingRegressor(random_state=0)
gbr = GradientBoostingRegressor(learning_rate=0.15, random_state=0)
# Fit the model on the training set
gbr.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
gbr_train_pred = gbr.predict(X_train)
gbr_val_pred = gbr.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_gbr = mean_squared_error(y_train, gbr_train_pred)
r2_train_gbr = r2_score(y_train, gbr_train_pred)
mae_train_gbr = mean_absolute_error(y_train, gbr_train_pred)
print("Train MSE: ", mse_train_gbr)
print("Train R2: ", r2_train_gbr)
print("Train MAE: ", mae_train_gbr)
# Calculate Forecast metrics on validation set
mse_val_gbr = mean_squared_error(y_validation, gbr_val_pred)
r2_val_gbr = r2_score(y_validation, gbr_val_pred)
mae_val_gbr = mean_absolute_error(y_validation, gbr_val_pred)
print("Validation MSE: ", mse_val_gbr)
print("Validation R2: ", r2_val_gbr)
print("Validation MAE: ", mae_val_gbr)

Train MSE:  0.00015774893315272882
Train R2:  0.9999774657024468
Train MAE:  0.010587745566071286
Validation MSE:  0.5182970210553074
Validation R2:  0.7654837579959607
Validation MAE:  0.6038017349451336


In [ ]:
# Define the model
gbr = GradientBoostingRegressor()
# Define the hyperparameters to be tuned
params = {
    'loss': ['squared_error'], # ['squared_error', 'absolute_error', 'huber', 'quantile']
    'learning_rate': [0.2], # [0.1] 
    'n_estimators': range(100, 200, 1),  # [110]
    'subsample': [1.0], 
    'criterion': ['friedman_mse'], # ['friedman_mse', 'squared_error']
    'min_samples_split': [2], 
    'min_samples_leaf': [1], 
    'min_weight_fraction_leaf': [0.0], 
    'max_depth': [4], # range(3,6,1)
    'min_impurity_decrease': [0.0], 
    'init': [None], 
    'random_state': [0], 
    'max_features': [None], 
    'alpha': [0.9], 
    'verbose': [0], 
    'max_leaf_nodes': [None], 
    'warm_start': [False], 
    'validation_fraction': [0.1], 
    'n_iter_no_change': [None], 
    'tol': [0.0001], 
    'ccp_alpha': [0.0]
}

# Create the GridSearchCV object
grid_gbr = GridSearchCV(gbr, params, cv=tscv5, scoring='neg_mean_squared_error', n_jobs=-1)
# Fit the model on the training set with GridSearchCV
grid_gbr.fit(X_train, y_train)
# Print the best hyperparameters
print('Best hyperparameters:', grid_gbr.best_params_)
# Use the best model to make predictions on the validation set
y_val_pred = grid_gbr.predict(X_validation)
# Compute the mean squared error of the predictions on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)
# Use the best model to make predictions on the test set
#y_test_pred_gbr = grid_gbr.predict(X_test)

c:\Users\osmar.bolivar\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_search.py:1114: RuntimeWarning: overflow encountered in square
  (array - array_means[:, np.newaxis]) ** 2, axis=1, weights=weights
c:\Users\osmar.bolivar\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


Best hyperparameters: {'learning_rate': 1.757510624854793, 'max_depth': 5, 'n_estimators': 100, 'random_state': 123}
Validation MSE:  0.2815678564215272
Validation R2:  -1.4652283831985624
Validation MAE:  0.37399708361903783


In [72]:
# Create a DataFrame with the feature importance values
feature_importance_gbr = pd.DataFrame({'Feature': X_train.columns, 'Importance': gbr.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_gbr = feature_importance_gbr.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_gbr.columns = ['feat_gbr', 'imp_gbr']
# Print the feature importance DataFrame
feature_importance_gbr.head(15)

,feat_gbr,imp_gbr
0,lag_1,0.694396
1,oil2_bol,0.100050
2,chicken_tj,0.067620
3,lemon_co,0.038119
4,rice_su,0.010053
5,sugar_tj,0.009246
6,sugar_lp,0.005842
7,ycorn_bol,0.005311
8,libor,0.004387
9,ycorn_sc,0.004357


### **3.4. RF**
Without tuning:
Validation MSE:  0.888
Validation R2:  0.598
Validation MAE:  0.836

In [79]:
#rf = RandomForestRegressor(random_state=0)
rf = RandomForestRegressor(criterion="absolute_error", max_depth=7, random_state=0)

# Fit the model to the training data and make predictions on the validation set
rf.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
rf_train_pred = rf.predict(X_train)
rf_val_pred = rf.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_rf = mean_squared_error(y_train, rf_train_pred)
r2_train_rf = r2_score(y_train, rf_train_pred)
mae_train_rf = mean_absolute_error(y_train, rf_train_pred)
print("Train MSE: ", mse_train_rf)
print("Train R2: ", r2_train_rf)
print("Train MAE: ", mae_train_rf)
# Calculate Forecast metrics on validation set
mse_val_rf = mean_squared_error(y_validation, rf_val_pred)
r2_val_rf = r2_score(y_validation, rf_val_pred)
mae_val_rf = mean_absolute_error(y_validation, rf_val_pred)
print("Validation MSE: ", mse_val_rf)
print("Validation R2: ", r2_val_rf)
print("Validation MAE: ", mae_val_rf)

Train MSE:  0.051367765953412234
Train R2:  0.9926621594231866
Train MAE:  0.1702821126455092
Validation MSE:  0.6605221081313929
Validation R2:  0.7011305173158032
Validation MAE:  0.7131762000275602


In [ ]:
# Define the Random Forest Regression model
rf_reg = RandomForestRegressor()

# Define the hyperparameters to tune
param_grid_rf = {
    'n_estimators': range(100, 300, 10),  # [100]
    'criterion': ['absolute_error'], 
    'max_depth': [7], # [None] 
    'min_samples_split': [2],  # range(2,20,1) 
    'min_samples_leaf': [1], 
    'min_weight_fraction_leaf': [0.0], 
    'max_features': [1.0], 
    'max_leaf_nodes': [None], 
    'min_impurity_decrease': [0.0], 
    'bootstrap': [True], 
    'oob_score': [False], 
    'n_jobs':[None], 
    'random_state': [0], 
    'verbose': [0], 
    'warm_start': [False], 
    'ccp_alpha': [0.0], 
    'max_samples': [None], 
    'monotonic_cst': [None]
}

# Define the GridSearchCV object
grid_rf_reg = GridSearchCV(estimator=rf_reg, param_grid=param_grid_rf, cv=tscv5, scoring='neg_mean_squared_error')
# Fit the GridSearchCV object to the training data
grid_rf_reg.fit(X_train, y_train)
# Extract the best hyperparameters and score
best_params = grid_rf_reg.best_params_
best_score = grid_rf_reg.best_score_
# Print the best hyperparameters found by GridSearchCV
print(f"Best hyperparameters: {best_params}")
print(f"Best score: {best_score}")
# Instantiate a new Random Forest Regression model using the best hyperparameters
rf_reg_best = RandomForestRegressor(**best_params)
# Fit the model to the training data and make predictions on the validation set
rf_reg_best.fit(X_train, y_train)
# Use the best model to make predictions on the validation set
y_val_pred = rf_reg_best.predict(X_validation)
# Compute the mean squared error of the predictions on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Use the best model to make predictions on the test set
#y_test_pred_rf = rf_reg_best.predict(X_test)

In [80]:
# Create a DataFrame with the feature importance values
feature_importance_rf = pd.DataFrame({'Feature': X_train.columns, 'Importance': rf.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_rf = feature_importance_rf.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_rf.columns = ['feat', 'imp_rf']
# Print the feature importance DataFrame
feature_importance_rf.head(30)

,feat,imp_rf
0,lag_1,0.502908
1,chicken_tj,0.037595
2,oil2_bol,0.037031
3,flour_lp,0.023477
4,sugar_tj,0.017904
5,veglard_bol,0.017858
6,sugar_or,0.015273
7,flour_cb,0.014165
8,sugar_lp,0.012498
9,sugar_po,0.008818


### **3.5. ET**   
Without tuning:
Validation MSE:  4.867
Validation R2:  -1.202
Validation MAE:  2.030

In [87]:
#et = ExtraTreesRegressor(random_state=0)
et = ExtraTreesRegressor(max_depth=5, bootstrap=True, oob_score=True, random_state=0)
# Fit the model to the training data and make predictions on the validation set
et.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
et_train_pred = et.predict(X_train)
et_val_pred = et.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_et = mean_squared_error(y_train, et_train_pred)
r2_train_et = r2_score(y_train, et_train_pred)
mae_train_et = mean_absolute_error(y_train, et_train_pred)
print("Train MSE: ", mse_train_et)
print("Train R2: ", r2_train_et)
print("Train MAE: ", mae_train_et)
# Calculate Forecast metrics on validation set
mse_val_et = mean_squared_error(y_validation, et_val_pred)
r2_val_et = r2_score(y_validation, et_val_pred)
mae_val_et = mean_absolute_error(y_validation, et_val_pred)
print("Validation MSE: ", mse_val_et)
print("Validation R2: ", r2_val_et)
print("Validation MAE: ", mae_val_et)

Train MSE:  0.07229305322182888
Train R2:  0.9896730003824971
Train MAE:  0.20554565559230062
Validation MSE:  4.272498008234536
Validation R2:  -0.9331968661922647
Validation MAE:  1.8815938973731743


In [ ]:
# Define the Extra Trees Regression model
et_reg = ExtraTreesRegressor()
# Define the hyperparameter grid to search over
param_grid = {
    'n_estimators': [100], # range(100, 300, 15)
     'criterion': ['squared_error'], 
     'max_depth':[None], 
     'min_samples_split':[2], 
     'min_samples_leaf':[1], 
     'min_weight_fraction_leaf':[0.0], 
     'max_features':[1.0], 
     'max_leaf_nodes':[None], 
     'min_impurity_decrease':[0.0], 
     'bootstrap':[True], # [False] 
     'oob_score':[True], # [False]
     'n_jobs':[None], 
     'random_state':[0], 
     'verbose':[0], 
     'warm_start':[False], 
     'ccp_alpha':[0.0], 
     'max_samples':[None],   
     'monotonic_cst':[None]
}

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(et_reg, param_grid=param_grid, cv=tscv5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
# Print the best hyperparameters and the corresponding validation score
print("Best hyperparameters: ", grid_search.best_params_)
#print("Validation score: ", grid_search.best_score_)
# Use the best model to make predictions on the validation set
best_et_reg = grid_search.best_estimator_
y_val_pred = best_et_reg.predict(X_validation)
# Compute the mean squared error of the predictions on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Use the best model to make predictions on the test set
#y_test_pred_et = best_et_reg.predict(X_test)

In [88]:
# Create a DataFrame with the feature importance values
feature_importance_et = pd.DataFrame({'Feature': X_train.columns, 'Importance': et.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_et = feature_importance_et.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_et.columns = ['feat', 'imp_et']
# Print the feature importance DataFrame
feature_importance_et.head(30)

,feat,imp_et
0,lag_1,0.206079
1,oil2_bol,0.064711
2,veglard_bol,0.060907
3,oil_bol,0.053352
4,chicken_tj,0.039476
5,flour_cb,0.030305
6,lag_2,0.029958
7,oil_cb,0.029144
8,veglard_sc,0.028968
9,sugar_or,0.024855


## **4. Report**

#### End